# Loading Required Datasets

In [ ]:
## Reading the datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

df1 = pd.read_pickle('covid_demo2') #Covid demographic data  
df2 = pd.read_pickle('demo+cases2') #demographic data+ covid-cases by zipcode

## only numeric data 
df3=pd.DataFrame(df1.drop(['lab_report_date'],axis=1)) #Covid demographic data 
df4=pd.DataFrame(df2.drop(['zipcode','Location','County',
                           'week_number','tests_cumulative', 'deaths_cumulative', 'cases_cumulative'],axis=1)) #demographic data+ covid-cases by zipcode

df3=df3.dropna().astype('int').reset_index()
df4=df4.astype('int')

total=df3.sum()
groups_cases=total[3:21]
groups_death=total[21:39]
groups_cases=pd.DataFrame(groups_cases).rename(columns={0:'total_number'})
groups_death=pd.DataFrame(groups_death).rename(columns={0:'total_number'})
groups_death['prob_death']=groups_death['total_number']/total[1]
groups_cases['prob_cases']=groups_cases['total_number']/total[0]

## sortting based on probability 
groups_death=groups_death.sort_values(by=['prob_death'],ascending=False)
groups_cases=groups_cases.sort_values(by=['prob_cases'],ascending=False)


## Creating the format that we want 
## datasets (Racial)
prob1=np.array([0.020698,0.037505,0.020833,0.097971,0.088107])
df11=pd.DataFrame(prob1) #probability
df12=df4.iloc[:,[3,4,6,8,9]] #population 

# datasets (Age)
prob2=np.array([0.038741,0.052464,0.067502,0.078672,0.077986,0.078980,0.072750,0.078768])
df21=pd.DataFrame(prob2) #probability
df22=df4.iloc[:,[12,13,14,15,16,17,18,19]] #population 

## dataset (gender)

prob3=np.array([0.060259,0.058617]) #male female 
df31=pd.DataFrame(prob3) #probability
df32=df4.iloc[:,[10,11]]#population 


# Fair-Diverse Model 

In [ ]:
## Optimization_problem

import docplex.mp.model as cplex
import math
def Fair_allocation(df1,df2,b,alpha):
    m=cplex.Model(name='Covid-Optimal Allocation')
    b=b #total vaccines available
    
    ## required data (J*I dataframes)
    set_I=np.arange(df1.shape[0]) # number of groups
    set_J=np.arange(df2.shape[0]) # number of Centers
    
    ## Decision Variable
    x_vars  = {(j): m.continuous_var(
                name='x_{0}'.format(j)) for j in set_J} #number of vaccines allocated to j
    f_var  =  m.continuous_var(name='f_var') #f gap
    d_var = m.continuous_var(name='d_var') #d gap
    
    s=r=df2.sum().sum() #scale param for both sides 
    
    ## Constraints
    c0=m.add_constraint(sum(x_vars[j] for j in set_J) == b) #Capacity
    
    #diversity
    for j in set_J:
        c1=m.add_constraint(
            s*(x_vars[j]/df2.sum(axis=1)[j]-
            sum(x_vars[j] for j in set_J)/df2.sum().sum()) <= s*d_var)
        
        
    for j in set_J:
        c2=m.add_constraint(
            s*(x_vars[j]/df2.sum(axis=1)[j]-
            sum(x_vars[j] for j in set_J)/df2.sum().sum()) >=- s*d_var)       
    a=sum(sum((df2.iloc[j,i]*df1.iloc[i,0])for i in set_I)for j in set_J)
    
    # Fairness
    for i in set_I:
        c3=m.add_constraint(
         r*(sum(x_vars[j]*df2.iloc[j,i]/(sum((df2.iloc[j,i]*df1.iloc[i,0])for i in set_I)*(df2.sum()[i]))
           for j in set_J)-
        sum(sum((x_vars[j]*df1.iloc[i,0]*df2.iloc[j,i])/(sum(df2.iloc[j,i]*df1.iloc[i,0] for i in set_I)*a)
        for j in set_J) for i in set_I))
            <= r* f_var
        )
        
    for i in set_I:
        c4=m.add_constraint(
         r*(sum(x_vars[j]*df2.iloc[j,i]/(sum((df2.iloc[j,i]*df1.iloc[i,0])for i in set_I)*(df2.sum()[i]))
           for j in set_J)-
        sum(sum((x_vars[j]*df1.iloc[i,0]*df2.iloc[j,i])/(sum(df2.iloc[j,i]*df1.iloc[i,0] for i in set_I)*a)
        for j in set_J) for i in set_I))
            >=- r*f_var
        )
        

    c1=df2.sum().sum()/b #scale the two sides 
    c2=a/b #scale the two sides 
    
    m.minimize(alpha*r*f_var*(c2)+(1-alpha)*s*d_var*(c1))

    ##solve
    ans=m.solve()
    
    df_vars=pd.DataFrame({'x':x_vars})
    df_vars['allocations'] = ans.get_values(df_vars.x)
    
    return ans[f_var],ans[d_var],df_vars,df_vars.allocations


# Binary-Search Algorithm 

In [ ]:
## Main algorithm 

def opt_solution(epi_f,epi_d,df1,df2,b,print_status):
    
    #initialization
    alpha_l=0 #lower bound 
    alpha_u=1 #upper bound 
    teta=0.0001  #thereshold 
    alpha_m=0 #med point 
    b=b  #total number of vaccines 
    i=0
    while(abs(alpha_u-alpha_l)>=teta): 
        
        alpha_m=(alpha_u+alpha_l)/2
        f,d,x_opt,x_opt_alloc=Fair_allocation(df1,df2,b,alpha_m)
        
        if(print_status==True):
            print("\n Iteration number",i," : F and D are:",f,d)
            print("\n current alpha range: ","(",alpha_l,alpha_u,")")
            print("\n alpha_f is: ",alpha_m)
            print(abs(alpha_u-alpha_l))
            
        if(d<=epi_d and f<=epi_f):
            print("\n Optimal Range Found")
            break 
        elif(d<=epi_d and f>epi_f):
            alpha_l=(alpha_l+alpha_m)/2
            
        elif(d>epi_d and f<=epi_f):
            alpha_u=(alpha_u+alpha_m)/2 
        else:
            print("\n No optimal Range")
            print('d and f are: ',d,f)
            break 
        i=i+1 
    return f,d,x_opt,x_opt_alloc 

# Rounding Algorithm

In [ ]:
## Proposed Rounding Algorithm 

def rounding(df,b):
    
    r1=int(b-df.loc[:,['Diverse-only']].sum())
    r2=int(b-df.loc[:,['Fair-Only']].sum())
    r3=int(b-df.loc[:,['alpha=0.5']].sum())
    r4=int(b-df.loc[:,['alpha_tuned']].sum())
  
    if(r1<0):
        df=df.sort_values(by=['total_pop'],ascending=False)
        for i in range((abs(r1))):
            df.loc[i,['Diverse-only']]=df.loc[i,['Diverse-only']]-1
    if(r1>0):
        df=df.sort_values(by=['total_pop'],ascending=False)
        for i in range((abs(r1))):
            df.loc[i,['Diverse-only']]=df.loc[i,['Diverse-only']]+1
            
    if(r2<0):
        df=df.sort_values(by=['total_pop'],ascending=False)
        for i in range((abs(r2))):
            df.loc[i,['Fair-Only']]=df.loc[i,['Fair-Only']]-1
    if(r2>0):
        df=df.sort_values(by=['Exposed_Population'],ascending=False)
        for i in range((abs(r2))):
            df.loc[i,['Fair-Only']]=df.loc[i,['Fair-Only']]+1
            
    if(r3<0):
        df=df.sort_values(by=['total_pop'],ascending=False)
        for i in range(abs(r3)):
            df.loc[i,['alpha=0.5']]=df.loc[i,['alpha=0.5']]-1
    if(r3>0):
        df=df.sort_values(by=['Exposed_Population'],ascending=False)
        for i in range((abs(r3))):
            df.loc[i,['alpha=0.5']]=df.loc[i,['alpha=0.5']]+1 
            
    if(r4<0):
        df=df.sort_values(by=['total_pop'],ascending=False)
        for i in range((abs(r4))):
            df.loc[i,['alpha_tuned']]=df.loc[i,['alpha_tuned']]-1
    if(r4>0):
        df=df.sort_values(by=['Exposed_Population'],ascending=False)
        for i in range((abs(r4))):
            df.loc[i,['alpha_tuned']]=df.loc[i,['alpha_tuned']]+1
            
                  
    return df 

# Racial instance Problem 

In [ ]:
b=200000 #total resources 

f1,d1,fair1,fair_alloc=Fair_allocation(df11,df12,b,0.5) # fair-diverse with alpha=0.5 
f2,d2,fair2,fair_alloc2=Fair_allocation(df11,df12,b,0)  # diversity-only 
f3,d3,x_opt,x_opt_alloc=opt_solution(0.025,0.025,df11,df12,b,False) # fair-diverse with tuned alpha range 
f4,d4,fair4,fair_alloc4=Fair_allocation(df11,df12,b,1)  #Fair-Only 

## Tabel 1 
fair1=pd.DataFrame(fair1).rename(columns={'allocations': "alpha=0.5"})
fair1['zipcode']=df2['zipcode']
fair1['total_pop']=df12.sum(axis=1)
fair1['Exposed_Population']=df12['Black Or African American']*0.037505+df12['White']*0.020698+df12['Asian']*0.020833+df12['Other Race']*0.097971+df12['Two Or More Races']*0.088107
fair1['Exposed_Population']=fair1.Exposed_Population.round(0)
fair1['alpha=0.5']=fair_alloc.round(0)
fair1['alpha_tuned']=x_opt_alloc.round(0)
fair1['Diverse-only']=fair_alloc2.round(0)
fair1['Fair-Only']=fair_alloc4.round(0)
fair1_tabel=fair1.loc[:,['zipcode','total_pop','Exposed_Population','alpha=0.5','alpha_tuned','Diverse-only','Fair-Only']].sort_values(by=['total_pop'],ascending=False)

## Remained/Excess Resources  

r1=b-fair1_tabel['Diverse-only'].sum()
r2=b-fair1_tabel['Fair-Only'].sum()
r3=b-fair1_tabel['alpha=0.5'].sum()
r4=b-fair1_tabel['alpha_tuned'].sum()

## Remained/Excess Resources after rounding 
r1=b-rounding(fair1_tabel,b)['Diverse-only'].sum()
r2=b-rounding(fair1_tabel,b)['Fair-Only'].sum()
r3=b-rounding(fair1_tabel,b)['alpha=0.5'].sum()
r4=b-rounding(fair1_tabel,b)['alpha_tuned'].sum()

#fair1_tabel.head(15)


In [ ]:
# Plotting the results 

import matplotlib.pyplot as plt

f1=fair1.sort_values(by=['total_pop'],ascending=False).head(15) #data 

plt.rcParams.update({'font.size': 28})
plt.figure(figsize=(15,14),dpi=100)
plt.plot( 'zipcode', 'Diverse-only', data=f1, marker='o', markerfacecolor='green',markersize=6, color='olive', linewidth=4)
plt.plot( 'zipcode', 'alpha=0.5', data=f1, marker='o', markerfacecolor='blue', markersize=6, color='skyblue', linewidth=4)
plt.plot( 'zipcode', 'alpha_tuned', data=f1, marker='o', markerfacecolor='red',markersize=6, color='pink', linewidth=4)
L=plt.legend(numpoints=1,loc='upper right',fontsize=27)
L.get_texts()[0].set_text('Diverse-only')
L.get_texts()[1].set_text('alpha=0.5')
L.get_texts()[2].set_text('Fair-Diverse(tuned alpha)')
plt.xlabel('Zipcodes')
plt.ylabel('Total Allocations')
plt.xticks(rotation=50)
plt.show()